In [54]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis,spinful_fermion_basis_1d,boson_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library
#
##### setting up parameters for simulation
# physical parameters
l = 1
L_f = 3 # system size f
L_b = L_f-1 # system size b
Nf, Nb = 2, l # number of fermions, bosons
N = Nf + Nb # total number of particles
tb, tf = 0.1, 0.1 # boson, fermon hopping strength
U = 1.0  # n(n-1) interaction strength
V = 1.0
mu_b, mu_f = -0.5, -0.5 # b, f chemical potential 
# define time-dependent perturbation
A = 0
Omega = 1.0
def drive(t,Omega):
    return np.sin(Omega*t)
drive_args=[Omega]
#
###### create the basis
# build the two bases to tensor together to a bose-fermi mixture
basis_b1=boson_basis_1d(L_b,Nb=[i for i in range(2*l*L_b+1)],sps=2*l+1) # boson basis
basis_b2=boson_basis_1d(L_b,Nb=[i for i in range(2*l*L_b+1)],sps=2*l+1) # boson basis
basis_f=spinful_fermion_basis_1d(L_f,Nf = [(0,0),(0,1),(1,0),(1,1),(2,0),(0,2),(1,2),(2,1),(2,2),(0,3),(3,0),(3,1),(1,3),(2,3),(3,2),(3,3)],double_occupancy=True)
basis=tensor_basis(basis_b1,basis_b2,basis_f) # BFM
#
print(basis_b1)
print(basis_f)
##### create model
# define site-coupling lists
hop_b = [[-tb,i,(i+1)] for i in range(L_b-1)] # b hopping
int_list_bb = [[U/2.0,i,i] for i in range(L_b)] # bb onsite interaction
int_list_bb_lin = [[-U/2.0,i] for i in range(L_b)] # bb interaction, linear term
# 
hop_f_right = [[-tf,i,(i+1)] for i in range(L_f-1)] # f hopping right
hop_f_left = [[tf,i,(i+1)] for i in range(L_f-1)] # f hopping left
drive_f = [[A*(-1.0)**i,i] for i in range(L_f)] # density staggered drive
#
chem_f = [[mu_f,i] for i in range(L_f)] #chemical potential for fermions
chem_b = [[mu_b,i] for i in range(L_b)] #chemical potential for bosons
#
int_list_bf = [[V,i,j] for i in range(L_b) for j in range(L_f)] # bf onsite interaction
int_list_bf_m = [[-V,i,j] for i in range(L_b) for j in range(L_f)] # bf onsite interaction

#int_list_bf_offsite = [[V,i,i+1] for i in range(L_b)] # bf onsite interaction

penalty_quad = [[1000.,i,i] for i in range(L_b)] #penalty in order to stay in the sector n_b + n_d = 2l
penalty_lin = [[-4000.*l,i] for i in range(L_b)]
penalty_const = [[1000.*4*l**2,i] for i in range(L_b)]
penalty_quad_2 = [[2000.,i,i] for i in range(L_b)]




# create resource Hamiltonian
bosons_hopping = [	
            ["+-|||", hop_b], # bosons hop left
            ["|+-||", hop_b], # bosons hop left

            ["-+|||", hop_b], # bosons hop right
            ["|-+||", hop_b], # bosons hop right
]

bosons_on_site_int = [
            ["n|||", int_list_bb_lin], # bb onsite interaction
            ["|n||", int_list_bb_lin], # bb onsite interaction

            ["nn|||", int_list_bb], # bb onsite interaction
            ["|nn||", int_list_bb], # bb onsite interaction
]
            #
fermions_hopping = [
            ["|||+-", hop_f_left], # fermions hop left
            ["||+-|", hop_f_left], # fermions hop left

            ["|||-+", hop_f_right], # fermions hop right
            ["||-+|", hop_f_right], # fermions hop right
]
            #
boson_fermion_int_b1 = [

            ["n||n|", int_list_bf], # bf onsite interaction

]

boson_fermion_int_b2 = [
            ["n|||n", int_list_bf_m], # bf onsite interaction

]

boson_fermion_int_d1 = [

            ["|n|n|", int_list_bf], # bf onsite interaction

]

boson_fermion_int_d2 = [

            ["|n||n", int_list_bf_m], # bf onsite interaction
]

            #
chem_potential_f = [
            ["|||n", chem_f], #chemical potential fermions
            ["||n|", chem_f], #chemical potential fermions
]
chem_potential_b = [
            ["n|||", chem_b],  #chemical potential bosons 
            ["|n||", chem_b],  #chemical potential bosons    

            ]
dynamic = [["|n",drive_f,drive,drive_args]] # drive couples to fermions only

############################################## target  hamiltonian
a = 0.1
e = 1.0
m = 0.3

bosonic_part_LGT = [[a*e**2/4.,i,i,i,i] for i in range(L_b)] # b^dagger*b^dagger*b*b
mass_f = [[m+1.0/a,i,i] for i in range(L_f)] # f mass term
m_mass_f = [[-m-1.0/a,i,i] for i in range(L_f)] # f mass term

interaction_part_LGT = [[1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction
m_interaction_part_LGT = [[-1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction

coherent_b = [[1.,i] for i in range(L_b)]
coherent_f = [[1.,i] for i in range(L_f)]


#
static_coh = [
    
    ["+|||",coherent_b],
    ["-|||",coherent_b],

    ["|+||",coherent_b],
    ["|-||",coherent_b],

    ["||+|",coherent_f],
    ["||-|",coherent_f],
    
    ["|||+",coherent_f],
    ["|||-",coherent_f],

]


static_mix = [
    
    ["+|||",coherent_b],
    ["-|||",coherent_b],

    ["|+||",coherent_b],
    ["|-||",coherent_b],

    ["||+|",coherent_f],
    ["||-|",coherent_f],
    
    ["|||+",coherent_f],
    ["|||-",coherent_f],
#introducing the penalty to the Hamiltonian
    ["nn|||",penalty_quad],
    ["|nn||",penalty_quad],
    ["n|n||",penalty_quad_2],
    ["n|||",penalty_lin],
    ["|n||",penalty_lin],
    ["I|||",penalty_const]

]

static_mass = [
            ["||+|-", mass_f], # f mass term
            ["||-|+", m_mass_f], # f mass term
]

static_target = [
            ["++--|||", bosonic_part_LGT], # b^dagger*b^dagger*b*b
            ["|++--||", bosonic_part_LGT], # b^dagger*b^dagger*b*b

            ["||+|-", mass_f], # f mass term
            ["||-|+", m_mass_f], # f mass term


            ["+|-|+|-", interaction_part_LGT], #boson-fermion interaction
            ["-|+|-|+", m_interaction_part_LGT], #boson-fermion interaction
#introducing the penalty to the Hamiltonian
            ["nn|||",penalty_quad],
            ["|nn||",penalty_quad],
            ["n|n||",penalty_quad_2],
            ["n|||",penalty_lin],
            ["|n||",penalty_lin],
            ["I|||",penalty_const]

]

#Schwinger bosons at 0th link
b_0 = [[+0.5*e,0]]
d_0 = [[+0.5*e,0]]

b_0_m = [[-0.5*e,0]]
d_0_m = [[-0.5*e,0]]

#Schwinger bosons at 1st link
b_1 = [[+0.5*e,1]]
d_1 = [[+0.5*e,1]]

b_1_m = [[-0.5*e,1]]
d_1_m = [[-0.5*e,1]]

#Fermions at 0th site
rho_0 = [[-e,0]]

#Fermions at 1st site
rho_1 = [[-e,1]]

#Fermions at 2nd site
rho_2 = [[-e,2]]

#Identity at 0th site
eee_0 = [[e,0]]

#Identity at 1st site
eee_1 = [[e,1]]

#Identity at 2nd site
eee_2 = [[e,2]]


static_gauss_law_0 = [
    
    ["n|||", b_0],
    ["|n||", d_0_m],
    ["||n|", rho_0],
    ["|||n", rho_0],
    ["||I|", eee_0]

    
]



static_gauss_law_1 = [
    
    ["n|||", b_0_m],
    ["|n||", d_0],
    ["n|||", b_1],
    ["|n||", d_1_m],
    ["||n|", rho_1],
    ["|||n", rho_1],
    ["||I|", eee_1]

]    
    

static_gauss_law_2 = [
    
    ["n|||", b_1_m],
    ["|n||", d_1],
    ["||n|", rho_2],
    ["|||n", rho_2],
    ["||I|", eee_2]

    
]





################################################
###### set up Hamiltonian and initial states
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
H_R6  = hamiltonian(bosons_hopping,dynamic,basis=basis,**no_checks)
H_R1  = hamiltonian(bosons_on_site_int,dynamic,basis=basis,**no_checks)
H_R5  = hamiltonian(fermions_hopping,dynamic,basis=basis,**no_checks)
H_R3  = hamiltonian(chem_potential_f,dynamic,basis=basis,**no_checks)
H_R4  = hamiltonian(chem_potential_b,dynamic,basis=basis,**no_checks)
H_Rb1  = hamiltonian(boson_fermion_int_b1,dynamic,basis=basis,**no_checks)
H_Rb2  = hamiltonian(boson_fermion_int_b2,dynamic,basis=basis,**no_checks)
H_Rd1  = hamiltonian(boson_fermion_int_d1,dynamic,basis=basis,**no_checks)
H_Rd2  = hamiltonian(boson_fermion_int_d2,dynamic,basis=basis,**no_checks)


Gauss_law_0 = hamiltonian(static_gauss_law_0, dynamic_list = [], basis=basis, **no_checks)
Gauss_law_1 = hamiltonian(static_gauss_law_1, dynamic_list = [], basis=basis, **no_checks)
Gauss_law_2 = hamiltonian(static_gauss_law_2, dynamic_list = [], basis=basis, **no_checks)




#H_BFM = hamiltonian(static,dynamic,basis=basis,**no_checks)
H_target = hamiltonian(static_target,dynamic,basis=basis,**no_checks)

H_mix = H_R5 + H_R6

H_mass = hamiltonian(static_mass,dynamic,basis=basis,**no_checks)

H_coh = hamiltonian(static_coh,dynamic,basis=basis,**no_checks)
H_mix = hamiltonian(static_mix,dynamic,basis=basis,**no_checks)

Gauss_quadrat_0 = Gauss_law_0*Gauss_law_0
Gauss_quadrat_1 = Gauss_law_1*Gauss_law_1
Gauss_quadrat_2 = Gauss_law_2*Gauss_law_2

reference states: 
array index   /   Fock state   /   integer repr. 
     0.         |2 2>           8  
     1.         |2 1>           7  
     2.         |2 0>           6  
     3.         |1 2>           5  
     4.         |1 1>           4  
     5.         |1 0>           3  
     6.         |0 2>           2  
     7.         |0 1>           1  
     8.         |0 0>           0  
reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |1 1 1>|1 1 1>           63  
      1.         |1 1 1>|1 1 0>           62  
      2.         |1 1 1>|1 0 1>           61  
      3.         |1 1 1>|1 0 0>           60  
      4.         |1 1 1>|0 1 1>           59  
      5.         |1 1 1>|0 1 0>           58  
      6.         |1 1 1>|0 0 1>           57  
      7.         |1 1 1>|0 0 0>           56  
      8.         |1 1 0>|1 1 1>           55  
      9.         |1 1 0>|1 1 0>           54  
     10.         |1 1 0>|1 0 1>           53  
     11.         |1 1

In [180]:

def exp_TmTdagger(liste):
    static_f1f1 = [[ liste[0],i,i] for i in range(L_f)]
    static_f1f2 = [[ liste[1],i,i] for i in range(L_f)]
    static_f2f1 = [[ -liste[1],i,i] for i in range(L_f)]
    static_f2f2 = [[ liste[2],i,i] for i in range(L_f)]

    static_bb = [[ liste[3],i,i] for i in range(L_b)]
    static_bd = [[ liste[4],i,i] for i in range(L_b)]
    static_db = [[ liste[4],i,i] for i in range(L_b)]
    static_dd = [[ liste[5],i,i] for i in range(L_b)]
    

    static_t = [

        ["||+-|",static_f1f1],
        ["||+|-",static_f1f2],
        ["||-|+",static_f2f1],
        ["|||+-",static_f2f2],

        ["+-|||",static_bb],
        ["+|-||",static_bd],
        ["-|+||",static_db],
        ["|+-||",static_dd],

    ]
    T = hamiltonian(static_t, dynamic_list = [], basis=basis, **no_checks)
    expon = T
    return exp_op(1j*expon)


def static(liste):
    static_f1f1 = [[ liste[0],i,i] for i in range(L_f)]
    static_f1f2 = [[ liste[1],i,i] for i in range(L_f)]
    static_f2f1 = [[ liste[2],i,i] for i in range(L_f)]
    static_f2f2 = [[ liste[3],i,i] for i in range(L_f)]

    static_bb = [[ liste[4],i,i] for i in range(L_b)]
    static_bd = [[ liste[5],i,i] for i in range(L_b)]
    static_db = [[ liste[6],i,i] for i in range(L_b)]
    static_dd = [[ liste[7],i,i] for i in range(L_b)]
    

    static_t = [

        ["||+-|",static_f1f1],
        ["||+|-",static_f1f2],
        ["||-|+",static_f2f1],
        ["|||+-",static_f2f2],

        ["+-|||",static_bb],
        ["+|-||",static_bd],
        ["-|+||",static_db],
        ["|+-||",static_dd],

    ]
    return static_t

lissst = np.array([1+0.0005*i*(-1)**i for i in range(6)])
print(np.size(lissst))
#exp_TmTdagger(lissst).__init__(static(lissst),dynamic = [],basis=basis, check_herm = True)
#for checking the Hermiticity of the operators

6


In [48]:
static_f1f1 = [[ 1.,i,i] for i in range(L_f)]
static_f1f2 = [[ 1.,i,i] for i in range(L_f)]
static_f2f1 = [[ -1.,i,i] for i in range(L_f)]
static_f2f2 = [[ 1.,i,i] for i in range(L_f)]

static_bb = [[ 1.,i,i] for i in range(L_b)]
static_bd = [[ 1.,i,i] for i in range(L_b)]
static_db = [[ 1.,i,i] for i in range(L_b)]
static_dd = [[ 1.,i,i] for i in range(L_b)]


static_t = [

    ["||+-|",static_f1f1],
    ["||+|-",static_f1f2],
    ["||-|+",static_f2f1],
    ["|||+-",static_f2f2],

    ["+-|||",static_bb],
    ["+|-||",static_bd],
    ["-|+||",static_db],
    ["|+-||",static_dd],

]
T = hamiltonian(static_t, dynamic_list = [], basis=basis, **no_checks)

T.__init__(static_t, dynamic_list = [],basis=basis, check_herm = True)


Hermiticity check passed!


<ipython-input-48-e8f481678e7c>:27: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_symm=False in hamiltonian
  T.__init__(static_t, dynamic_list = [],basis=basis, check_herm = True)
<ipython-input-48-e8f481678e7c>:27: UserWarning: Test for particle conservation not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_pcon=False in hamiltonian
  T.__init__(static_t, dynamic_list = [],basis=basis, check_herm = True)


In [47]:
t = T.toarray(0)
print(t)

[[13.987+0.j  0.   +0.j  0.   +0.j ...  0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j 12.986+0.j  0.   +0.j ...  0.   +0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j 12.986+0.j ...  0.   +0.j  0.   +0.j  0.   +0.j]
 ...
 [ 0.   +0.j  0.   +0.j  0.   +0.j ...  1.001+0.j  0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j ...  0.   +0.j  1.001+0.j  0.   +0.j]
 [ 0.   +0.j  0.   +0.j  0.   +0.j ...  0.   +0.j  0.   +0.j  0.   +0.j]]


In [49]:
S = exp_op(T,a=1j)
smat = S.get_mat()
mat_arr = smat.toarray()

In [50]:
mat_arr_conj = mat_arr.conj().T

a = np.dot(mat_arr_conj,mat_arr)
print(a)

[[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 1.+0.j]]


In [51]:
psi_0 = np.zeros(basis.Ns) # allocate space for state
#1.
i_0 = basis.index("21","01","101","100")
i_1 = basis.index("21","01","100","101")

#2.
i_2 = basis.index("10","12","101","001")
i_3 = basis.index("10","12","001","101")

#3.
i_4 = basis.index("22","00","110","100")
i_5 = basis.index("22","00","100","110")

#4.
i_6 = basis.index("01","21","011","010")
i_7 = basis.index("01","21","010","011")

#5.
i_8 = basis.index("12","10","110","010")
i_9 = basis.index("12","10","010","110")

#6.
i_10 = basis.index("00","22","011","001")
i_11 = basis.index("00","22","001","011")

psi_0[i_0] =1./np.sqrt(8)
psi_0[i_1] = -1./np.sqrt(8)
psi_0[i_2] = 1./np.sqrt(8)
psi_0[i_3] =-1./np.sqrt(8)
#psi_0[i_4] = 1./np.sqrt(12)
#psi_0[i_5] = -1./np.sqrt(12)
psi_0[i_6] =1./np.sqrt(8)
psi_0[i_7] = -1./np.sqrt(8)
psi_0[i_8] = 1./np.sqrt(8)
psi_0[i_9] =-1./np.sqrt(8)
#psi_0[i_10] = 1./np.sqrt(12)
#psi_0[i_11] = -1./np.sqrt(12)

print(np.dot(psi_0,psi_0))
print(Gauss_law_0.expt_value(psi_0))
print(Gauss_law_1.expt_value(psi_0))
print(Gauss_law_2.expt_value(psi_0))
print(H_target.expt_value(psi_0))

0.9999999999999998
0j
0j
0j
(10.35000000000018+0j)


In [53]:
psi = mat_arr@psi_0
print(psi_0)
print(psi)
print(np.dot(psi_0,psi_0))
print(np.dot(psi,psi))

[0. 0. 0. ... 0. 0. 0.]
[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
0.9999999999999998
(0.10840400674667286+0.032589739592626815j)


In [188]:
sconj = S.getH()
print(np.dot(sconj.get_mat(),S.get_mat()))


  (0, 0)	(-0.9552377669095268-0.29583916013554185j)
  (8, 1)	(-0.6775685561900717-0.6070242288181752j)
  (1, 1)	(-0.276736658257406+0.3095791647813615j)
  (16, 2)	(-0.6775685561900717-0.6070242288181752j)
  (2, 2)	(-0.276736658257406+0.3095791647813615j)
  (24, 3)	(-0.3299219544388184-0.758970438343919j)
  (17, 3)	(-0.3462662663564309+0.15097222656066578j)
  (10, 3)	(0.3462662663564309-0.15097222656066572j)
  (3, 3)	(0.06908434401993238+0.15797737272289702j)
  (32, 4)	(-0.6775685561900717-0.6070242288181752j)
  (4, 4)	(-0.276736658257406+0.3095791647813615j)
  (40, 5)	(-0.3299219544388184-0.758970438343919j)
  (33, 5)	(-0.3462662663564309+0.15097222656066578j)
  (12, 5)	(0.3462662663564309-0.15097222656066572j)
  (5, 5)	(0.06908434401993238+0.15797737272289702j)
  (48, 6)	(-0.3299219544388184-0.758970438343919j)
  (34, 6)	(-0.3462662663564309+0.15097222656066578j)
  (20, 6)	(0.3462662663564309-0.15097222656066572j)
  (6, 6)	(0.06908434401993238+0.15797737272289702j)
  (56, 7)	(0.015167

In [168]:
t = T.toarray(0)
T_mat_exp = expm(1j*t)


In [169]:
exp_conj = np.matrix.getH(T_mat_exp)
print(np.dot(exp_conj, T_mat_exp))

[[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 1.+0.j]]


In [185]:
psi = T.evolve(psi_0,0,1)
print(np.dot(psi,psi))

(0.1097292553171917-0.031320040729015335j)


0.9999999999999998
0j
0j
0j
(10.35000000000018+0j)


In [133]:
#print(H_target.expt_value(S.dot(psi_0)))
print(H_target.expt_value(psi_0))
#print(np.dot(S.dot(psi_0),psi_0))

(10.35000000000018+0j)


[0. 0. 0. ... 0. 0. 0.]
[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
0.9999999999999998
(0.10972929465110222+0.03132005872189506j)


In [ ]:

def extract_energy(a):
    psi_0 = np.array([complex(0) for i in range(basis.Ns)]) # allocate space for state
    i_0 = basis.index("11","11","000","000")
    cost_function = np.real(H_target.expt_value(psi_0)) + 1000*(np.real(Gauss_quadrat_0.expt_value(psi_0))+np.real(Gauss_quadrat_1.expt_value(psi_0))+np.real(Gauss_quadrat_2.expt_value(psi_0)))
    return cost_function


from scipy.optimize import brute
from scipy.optimize import minimize
import datetime
brute_ranges = (slice(-5.,5.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.))
resbrute = 0
resopt = 0
print(datetime.datetime.now())
resbrute = brute(extract_energy, brute_ranges ,full_output = True)
print(datetime.datetime.now())
resopt = minimize(fun = extract_energy,x0 = resbrute[0],method = "Nelder-Mead" ,tol = 1e-8)
print(datetime.datetime.now())
print(resopt.x)
print(resopt.fun)

In [2]:
eigenvalues, eigenvectors = H_target.eigh()

In [14]:
gauss_law = Gauss_law_2.dot(eigenvectors[:,0])
for i in range(len(gauss_law)):
    if np.abs(gauss_law[i]) >= 1e-3:
        print(i)
        print(gauss_law[i])
    if eigenvectors[i][0] >= 1e-5:
        print(eigenvectors[i][0])
        print(i)

(0.10121548771756084+0j)
523
(0.02912872723883702+0j)
537
(0.1348156618495796+0j)
1043
(0.16612423347744248+0j)
1050
(0.3107243144668309+0j)
2574
(0.3107243144668274+0j)
2595
(0.2814102412385909+0j)
2602
(0.3536537458630974+0j)
2616
(0.1661242334774395+0j)
4133
(0.13481566184958158+0j)
4140


In [137]:
psi_0 = np.zeros(basis.Ns) # allocate space for state
i_0 = basis.index("21","01","101","100")
i_1 = basis.index("21","01","100","101")


i_2 = basis.index("10","12","101","001")
i_3 = basis.index("10","12","001","101")


i_4 = basis.index("22","00","110","100")
i_5 = basis.index("22","00","100","110")


psi_0[i_0] =1./np.sqrt(6)
psi_0[i_1] = -1./np.sqrt(6)
psi_0[i_2] = 1./np.sqrt(6)
psi_0[i_3] =-1./np.sqrt(6)
psi_0[i_4] = 1./np.sqrt(6)
psi_0[i_5] = -1./np.sqrt(6)
print(Gauss_law_0.expt_value(psi_0))
print(Gauss_law_1.expt_value(psi_0))
print(Gauss_law_2.expt_value(psi_0))
print(H_target.expt_value(psi_0))
print(np.dot(psi_0,psi_0))

0j
0j
0j
(13.700000000000246+0j)
1.0000000000000002


In [12]:
a = np.array([1.,2.,3.])
psi_0 = np.zeros(basis.Ns) # allocate space for state
psi_1 = np.zeros(basis.Ns) # allocate space for state
psi_2 = np.zeros(basis.Ns) # allocate space for state

i_0 = basis.index("21","01","101","100")
i_1 = basis.index("10","12","101","001")
i_2 = basis.index("22","00","110","100")

psi_0[i_0] =1.
psi_1[i_1] = 1.
psi_2[i_2] = 1.

print(np.dot(psi_0,psi_0))
print(np.dot(psi_1,psi_1))
print(np.dot(psi_2,psi_2))
print(np.real(H_target.expt_value(psi_2)) + 1000*(np.real(Gauss_quadrat_0.expt_value(psi_2))+np.real(Gauss_quadrat_1.expt_value(psi_2))+np.real(Gauss_quadrat_2.expt_value(psi_2))))

1.0
1.0
1.0
0.1000000000003638


In [9]:
def extract_energy(a):
    psi_0 = np.array([complex(0) for i in range(basis.Ns)]) # allocate space for state
    i_0 = basis.index("11","11","000","000")
    cost_function = np.real(H_target.expt_value(psi_0)) + 1000*(np.real(Gauss_quadrat_0.expt_value(psi_0))+np.real(Gauss_quadrat_1.expt_value(psi_0))+np.real(Gauss_quadrat_2.expt_value(psi_0)))
    return cost_function


from scipy.optimize import brute
from scipy.optimize import minimize
import datetime
brute_ranges = (slice(-5.,5.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.))
resbrute = 0
resopt = 0
print(datetime.datetime.now())
resbrute = brute(extract_energy, brute_ranges ,full_output = True)
print(datetime.datetime.now())
resopt = minimize(fun = extract_energy,x0 = resbrute[0],method = "Nelder-Mead" ,tol = 1e-8)
print(datetime.datetime.now())
print(resopt.x)
print(resopt.fun)

2021-12-14 10:58:18.393008


<ipython-input-9-ec106da6d276>:7: RuntimeWarning: invalid value encountered in cdouble_scalars
  psi_0[i_0] =(a[0]+1j*a[1])/np.sqrt(a[0]**2+a[1]**2+a[2]**2+a[3]**2+a[4]**2+a[5]**2)
<ipython-input-9-ec106da6d276>:8: RuntimeWarning: invalid value encountered in cdouble_scalars
  psi_0[i_1] = (a[2]+1j*a[3])/np.sqrt(a[0]**2+a[1]**2+a[2]**2+a[3]**2+a[4]**2+a[5]**2)
<ipython-input-9-ec106da6d276>:9: RuntimeWarning: invalid value encountered in cdouble_scalars
  psi_0[i_2] = (a[4]+1j*a[5])/np.sqrt(a[0]**2+a[1]**2+a[2]**2+a[3]**2+a[4]**2+a[5]**2)


2021-12-14 11:13:33.573151
2021-12-14 11:13:33.661086
[ 6.25655088e-05  6.23050905e-05  6.25655088e-05  6.25655088e-05
 -5.14431918e-13 -1.68695814e-12]
0.050000000000181885


In [2]:
ei,eiv = H_coh.eigh()
psi_0 = eiv[:,0]

In [49]:
H_ini = H_mix + 1e3*(Gauss_quadrat_0 + Gauss_quadrat_1 + Gauss_quadrat_2)
ei,eiv = H_mix.eigh()
psi_0 = eiv[:,0]

In [36]:
print(ei[0])

-0.013999964992060968


In [75]:
print(np.dot(psi_0,psi_0))

0.9999999999999996


In [82]:
eigval,eigvec = H_mix.eigh()
psi_0 = eigvec[:,0]

In [ ]:
#the bare vacuum is the 40th eigenvector of H_mass!

In [80]:
print(H_target.expt_value(psi_0))

(8.881784197001252e-16+0j)


In [35]:
com = commutator(H_R1, H_R2)
#print(com)
com_second = commutator(H_R6,com)
com_third = commutator(H_R6,com_second)
com_fourth = commutator(H_R6,com_third)
com_fifth = commutator(H_R6,com_fourth)
com_sixth = commutator(H_R6,com_fifth)
com_seventh = commutator(H_R6,com_sixth)

#print(com_seventh)
print(com)

static mat: 



dynamic:



In [11]:
psi_0 = np.zeros(basis.Ns) # allocate space for state
i_0 = basis.index("11","11","110","001")
i_1 = basis.index("22","00","110","100")
i_2 = basis.index("00","22","011","001")

psi_0[i_0] = -1./2
psi_0[i_1] = 1./np.sqrt(6)
psi_0[i_2] = np.sqrt(7)/np.sqrt(12)

In [5]:
psi_a = np.zeros(basis.Ns)
psi_b = np.zeros(basis.Ns)
psi_a[i_0] = 1.
psi_b[i_1] = 1.
print(H_target.expt_value(psi_a))
print(H_target.expt_value(psi_b))
print(H_target.expt_value(psi_0))

0j
(0.1000000000003638+0j)
(0.07500000000027285+0j)


In [5]:
eigenvalues, eigenvectors = H_target.eigh() 

In [10]:
print(eigenvalues[0])

-35.690081555671895


In [39]:
gauss_law = Gauss_law_2.dot(psi_0)
for i in range(len(gauss_law)):
    if np.abs(gauss_law[i]) >= 1e-3:
        print(i)
        print(gauss_law[i])

In [7]:
for i in range(len(eigenvalues)):
    if (np.abs(Gauss_law_0.expt_value(eigenvectors[:,i]))<=1e-8 and 
        np.abs(Gauss_law_1.expt_value(eigenvectors[:,i]))<=1e-8 and
        np.abs(Gauss_law_2.expt_value(eigenvectors[:,i]))<=1e-8):
        print(i," with eigenvalue:", eigenvalues[i])
        

0  with eigenvalue: -35.690081555671895
25  with eigenvalue: -26.255814330486572
63  with eigenvalue: -18.233420409562463
112  with eigenvalue: -12.67532050147576
140  with eigenvalue: -10.24961044216821
232  with eigenvalue: -4.6351163184185395
243  with eigenvalue: -0.8585040415947308
332  with eigenvalue: 0.9347518315496658
348  with eigenvalue: 4.768292700699911
417  with eigenvalue: 10.349602336459215
454  with eigenvalue: 12.726987679893101
517  with eigenvalue: 18.36475921447891
550  with eigenvalue: 26.34053230889466
575  with eigenvalue: 35.712941527403544
3136  with eigenvalue: 3966.805850539433
3137  with eigenvalue: 3966.8058505394347
3138  with eigenvalue: 3966.9058505394314
3139  with eigenvalue: 3966.9058505394323
3204  with eigenvalue: 3979.724968750049
3205  with eigenvalue: 3979.724968750049
3206  with eigenvalue: 3979.8249687500493
3207  with eigenvalue: 3979.8249687500493
3264  with eigenvalue: 3987.405850539434
3265  with eigenvalue: 3987.405850539434
3266  with ei

In [24]:
#print(Gauss_law_0.expt_value(eigenvectors[:,0]))
#print(Gauss_law_1.expt_value(eigenvectors[:,0]))
#print(Gauss_law_2.expt_value(eigenvectors[:,0]))
psi_0 = eiv[:,0]
print(Gauss_law_0.expt_value(psi_0))
print(Gauss_law_1.expt_value(psi_0))
print(Gauss_law_2.expt_value(psi_0))

(0.09285498883973788+0j)
(-0.09285335486723861+0j)
(0.09285498883957403+0j)


In [12]:
import datetime
print(datetime.datetime.now())
print(extract_energy(np.array([1.,2.,3.,4.]),psi_0,1))
print(datetime.datetime.now())

2021-12-13 17:04:21.312927
2486.6103781779175
2021-12-13 17:04:21.344337


In [11]:
#psi_0 = eigenvectors_R1[:,0]

psi_t0 = H_R1.evolve(psi_0,0.0,10.)
psi_t1 = H_R2.evolve(psi_t0,0.0,10.)
psi_t2 = H_R3.evolve(psi_t1,0.0,10.)
psi_t3 = H_R4.evolve(psi_t2,0.0,10.)
psi_t4 = H_R5.evolve(psi_t3,0.0,10.)
psi_t5 = H_R6.evolve(psi_t4,0.0,10.)
psi_t6 = H_R1.evolve(psi_t5,0.0,10.)

print(np.real(H_target.expt_value(psi_t6)))
#print(Gauss_law_1.expt_value(psi_0))
#print(Gauss_law_1.expt_value(psi_t2))
#print(np.real(H_target.expt_value(eigenvectors[:,0]/np.sqrt(2)+eigenvectors[:,1]/np.sqrt(2))))
#print(np.dot(psi_0,psi_t3))
print(np.real(H_target.expt_value(psi_0)))


485.3920979321217
0.06666666666690922


In [6]:
Gauss_quadrat_0 = Gauss_law_0*Gauss_law_0
Gauss_quadrat_1 = Gauss_law_1*Gauss_law_1
Gauss_quadrat_2 = Gauss_law_2*Gauss_law_2

In [26]:
def extract_energy(theta,psi,a):
    psi_t0 = H_R1.evolve(psi,0.0,theta[0])
    psi_t1 = H_Rb1.evolve(psi_t0,0.0,theta[1])
    psi_t2 = H_Rb2.evolve(psi_t1,0.0,theta[2])
    psi_t3 = H_Rd1.evolve(psi_t2,0.0,theta[3])
    psi_t4 = H_Rd2.evolve(psi_t3,0.0,theta[4])
    psi_t5 = H_R5.evolve(psi_t4,0.0,theta[5])
    psi_t6 = H_R6.evolve(psi_t5,0.0,2*theta[6])
    psi_t7 = H_R5.evolve(psi_t6,0.0,theta[5])
    psi_t8 = H_Rb1.evolve(psi_t7,0.0,theta[1])
    psi_t9 = H_Rb2.evolve(psi_t8,0.0,theta[2])
    psi_t10 = H_Rd1.evolve(psi_t9,0.0,theta[3])
    psi_t11 = H_Rd2.evolve(psi_t10,0.0,theta[4])
    psi_t12 = H_R1.evolve(psi_t11,0.0,theta[0])



    #cost_function = np.real(H_R5.expt_value(psi_t8)) + np.real(H_R6.expt_value(psi_t8))
    cost_function = np.real(H_target.expt_value(psi_t12)) + 1000*(np.real(Gauss_quadrat_0.expt_value(psi_t12))+np.real(Gauss_quadrat_1.expt_value(psi_t12))+np.real(Gauss_quadrat_2.expt_value(psi_t12)))
    return cost_function

def extract_state(theta,psi):
    psi_t0 = H_R1.evolve(psi,0.0,theta[0])
    psi_t1 = H_Rb1.evolve(psi_t0,0.0,theta[1])
    psi_t2 = H_Rb2.evolve(psi_t1,0.0,theta[2])
    psi_t3 = H_Rd1.evolve(psi_t2,0.0,theta[3])
    psi_t4 = H_Rd2.evolve(psi_t3,0.0,theta[4])
    psi_t5 = H_R5.evolve(psi_t4,0.0,theta[5])
    psi_t6 = H_R6.evolve(psi_t5,0.0,2*theta[6])
    psi_t7 = H_R5.evolve(psi_t6,0.0,theta[5])
    psi_t8 = H_Rb1.evolve(psi_t7,0.0,theta[1])
    psi_t9 = H_Rb2.evolve(psi_t8,0.0,theta[2])
    psi_t10 = H_Rd1.evolve(psi_t9,0.0,theta[3])
    psi_t11 = H_Rd2.evolve(psi_t10,0.0,theta[4])
    psi_t12 = H_R1.evolve(psi_t11,0.0,theta[0])
    
    psi_final = psi_t12
    
    return psi_final


NN = 3 #number of iterations
res_theta_0 = np.zeros(NN+1)
res_theta_1 = np.zeros(NN+1)
res_theta_2 = np.zeros(NN+1)
res_theta_3 = np.zeros(NN+1)
res_theta_4 = np.zeros(NN+1)
res_theta_5 = np.zeros(NN+1)
res_theta_6 = np.zeros(NN+1)



res_theta_0[0] = 0.
res_theta_1[0] = 0.
res_theta_2[0] = 0.
res_theta_3[0] = 0.
res_theta_4[0] = 0.
res_theta_5[0] = 0.
res_theta_6[0] = 0.


psi_ini = np.array([[complex(0.) for i in range(basis.Ns)]for j in range(NN+1)])
psi_ini[0] = psi_0

res_energy = np.zeros(NN+1)
res_energy[0] = extract_energy(np.zeros(7),psi_0,1)
#res_energy[0] = np.real(H_R5.expt_value(psi_ini[0]))+np.real(H_R6.expt_value(psi_ini[0]))



res_gauss_law = np.array([complex(0.) for i in range(NN+1)])
res_gauss_law[0] = np.real(Gauss_quadrat_0.expt_value(psi_ini[0])) + np.real(Gauss_quadrat_1.expt_value(psi_ini[0])) + np.real(Gauss_quadrat_2.expt_value(psi_ini[0]))

#print(extract_state(np.array([res_theta_0[0],res_theta_1[0],res_theta_2[0]]),psi_ini[0]))
#print(res_gauss_law)
#print(np.dot(extract_state(np.array([res_theta_0[0],res_theta_1[0],res_theta_2[0]]),psi_ini[0]),psi_0))

In [ ]:
from scipy.optimize import brute
from scipy.optimize import minimize
import datetime
brute_ranges = (slice(-0.02,10.,2.),slice(-0.02,10.,2.),slice(-0.02,10.,2.),slice(-0.02,10.,2.),slice(-0.02,10.,2.),slice(-0.02,10.,2.),slice(-0.02,10.,2.))
for i in range(NN):
    resbrute = 0
    resopt = 0
    print(datetime.datetime.now())
    resbrute = brute(extract_energy, brute_ranges, args = (psi_ini[i],1) ,full_output = True)
    print(datetime.datetime.now())
    resopt = minimize(fun = extract_energy,x0 = resbrute[0],args = (psi_ini[i],1),method = "Nelder-Mead" ,tol = 1e-8)
    print(datetime.datetime.now())
    res_theta_0[i+1] = resopt.x[0]
    res_theta_1[i+1] = resopt.x[1]
    res_theta_2[i+1] = resopt.x[2]
    res_theta_3[i+1] = resopt.x[3]


    res_energy[i+1] = resopt.fun
    
    psi_ini[i+1] = extract_state(resopt.x,psi_ini[i])
    res_gauss_law[i+1] = (Gauss_law_0.expt_value(extract_state(resopt.x,psi_ini[i])))**2 + (Gauss_law_1.expt_value(extract_state(resopt.x,psi_ini[i])))**2 + (Gauss_law_2.expt_value(extract_state(resopt.x,psi_ini[i])))**2


2021-12-16 16:19:11.696731


In [140]:
#N=15, psi_0 is the ground state of mix+
print(res_energy)
#print(res_gauss_law)
#print(res_theta_0)
#print(res_theta_1)
#print(res_theta_2)
#print(res_theta_3)

[13.7       -8.4166647 -8.4166647 -8.4166647]


In [ ]:
#N=20, psi_0 is the ground state of mix++
print(res_energy)
#print(res_gauss_law)
#print(res_theta_0)
#print(res_theta_1)
#print(res_theta_2)
#print(res_theta_3)

In [15]:
#N=15, psi_0 is the ground state of mix+
print(res_energy)
#print(res_gauss_law)
#print(res_theta_0)
#print(res_theta_1)
#print(res_theta_2)
#print(res_theta_3)

[ 10.35      -10.2499997 -10.2499997 -10.2499997]


In [34]:
print(H_target.expt_value(psi_ini[0]))
print(H_target.expt_value(psi_ini[10]))

(6.228882153892714+0j)
(7.374715206103696-1.3931997916438732e-16j)


In [29]:
Ham = H_R5 + H_R6
eigenvalues_ham, eigenvectors_ham = Ham.eigh()
print(eigenvalues_ham[0])

-0.6828427124746194


In [57]:
psi_t0 = H_R1.evolve(psi_ini[2],0.0,res_theta_0[3])
psi_t1 = H_R2.evolve(psi_t0,0.0,res_theta_1[3])
psi_t2 = H_R5.evolve(psi_t1,0.0,res_theta_2[3])
psi_t3 = H_R6.evolve(psi_t2,0.0,2*res_theta_3[3])
psi_t4 = H_R5.evolve(psi_t3,0.0,res_theta_2[3])
psi_t5 = H_R2.evolve(psi_t4,0.0,res_theta_1[3])
psi_t6 = H_R1.evolve(psi_t5,0.0,res_theta_0[3])
print(H_target.expt_value(psi_t6))
print(extract_energy(np.array([0.,0.,0.,0.]),psi_ini[0],1))

(1.3802785701416546e-13+0j)
3000.0


In [33]:
Hamiltonian = H_R5 + H_R6
eigv,eigvec = Hamiltonian.eigh()
print(eigv[0])

-0.6828427124746194


In [62]:
from scipy.optimize import brute
from scipy.optimize import minimize
import datetime
brute_ranges = (slice(-5.,5.,1.),slice(-20.,20.,1.),slice(-5.,5.,1.),slice(-5.,5.,1.))
for i in range(NN):
    print(datetime.datetime.now())
    resbrute = 0
    resbrute = brute(extract_energy, brute_ranges, args = (psi_ini[i],1) ,full_output = True ,finish = minimize)
    res_theta_0[i+1] = resbrute[0][0]
    res_theta_1[i+1] = resbrute[0][1]
    res_theta_2[i+1] = resbrute[0][2]
    res_theta_3[i+1] = resbrute[0][3]

    res_energy[i+1] = resbrute[1]
    
    psi_ini[i+1] = extract_state(np.array([resbrute[0][0],resbrute[0][1],resbrute[0][2],resbrute[0][3]]),psi_ini[i])
    res_gauss_law[i+1] = Gauss_law_0.expt_value(extract_state(np.array([resbrute[0][0],resbrute[0][1],resbrute[0][2],resbrute[0][3]]),psi_ini[i]))**2 + Gauss_law_1.expt_value(extract_state(np.array([resbrute[0][0],resbrute[0][1],resbrute[0][2],resbrute[0][3]]),psi_ini[i]))**2 + Gauss_law_2.expt_value(extract_state(np.array([resbrute[0][0],resbrute[0][1],resbrute[0][2],resbrute[0][3]]),psi_ini[i]))**2
    print(datetime.datetime.now())

2021-11-11 15:20:29.362881


KeyboardInterrupt: 

In [141]:
import datetime

print(datetime.datetime.now())
extract_energy(np.array([8.1285,3.433,4.467,3.]))
print(datetime.datetime.now())

2021-10-25 14:44:18.600042
2021-10-25 14:44:18.827797


In [143]:
from scipy.optimize import brute
from scipy.optimize import minimize


brute_ranges = (slice(0,10.,1.),slice(0,10.,1.),slice(0,10.,1.),slice(0,10.,1.))
print(datetime.datetime.now())

resbrute = brute(extract_energy, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())

2021-10-25 14:44:31.443164
2021-10-25 15:25:08.030979


In [144]:
print(resbrute[0]) #theta1, theta2

[8. 9. 9. 9.]


In [145]:
print(resbrute[1]) #minimized energy

0.09999988539430749


In [146]:
Gauss_law.expt_value(psi_0)

0j

In [147]:
times1=np.linspace(0.0,restbrute[0][0],100)
times2=np.linspace(0.0,restbrute[0][1],100)
times3=np.linspace(0.0,restbrute[0][2],100)
times4=np.linspace(0.0,restbrute[0][3],100)


psi_t1 = H_R1.evolve(psi_ini,0.0,times1)
psi_t2 = H_R2.evolve([row[-1] for row in psi_t1],0.0,times2)
psi_t3 = H_R3.evolve([row[-1] for row in psi_t2],0.0,times3)
psi_t4 = H_R4.evolve([row[-1] for row in psi_t3],0.0,times4)

#print(np.abs(np.dot(psi_ground,np.array([row[-1] for row in psi_t1]))))
#print(np.abs(np.dot(np.array([row[-1] for row in psi_t1]),np.array([row[-1] for row in psi_t2]))))
#print(np.abs(np.dot(np.array([row[-1] for row in psi_t2]),np.array([row[-1] for row in psi_t3]))))
H_target.expt_value([row[-1] for row in psi_t4])

NameError: name 'restbrute' is not defined

In [163]:
H_coh.__init__(static_coh, dynamic_list = [],basis=basis, check_herm = True)
#for checking the Hermiticity of the operators
print('target')

Hermiticity check passed!
target


<ipython-input-163-423cababa137>:1: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_symm=False in hamiltonian
  H_coh.__init__(static_coh, dynamic_list = [],basis=basis, check_herm = True)
<ipython-input-163-423cababa137>:1: UserWarning: Test for particle conservation not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_pcon=False in hamiltonian
  H_coh.__init__(static_coh, dynamic_list = [],basis=basis, check_herm = True)


In [57]:
eigenvalues,eigenvectors = H_target.eigh()
#psi_ground = eigenvectors[:,0]
print(eigenvalues[0])

-38.04689509100519


In [109]:
from scipy.optimize import minimize
import datetime
print(datetime.datetime.now())
initial_guess = np.array([4.,8.,120.,3.])
nelder_mead = minimize(extract_energy, initial_guess,method='Nelder-Mead')
print(datetime.datetime.now())


2021-10-25 14:31:08.978682
2021-10-25 14:32:02.595452


In [110]:
nelder_mead.x

array([  3.99383535,   8.03443554, 119.68061779,   3.09978688])

In [111]:
nelder_mead.fun

0.09999972475591133

In [112]:
nelder_mead.nit

78

In [135]:
times1=np.linspace(0.0,nelder_mead.x[0],100)
times2=np.linspace(0.0,nelder_mead.x[1],100)
times3=np.linspace(0.0,nelder_mead.x[2],100)
times4=np.linspace(0.0,nelder_mead.x[3],100)


psi_t1 = H_R1.evolve(psi_ini,0.0,times1)
psi_t2 = H_R2.evolve([row[-1] for row in psi_t1],0.0,times2)
psi_t3 = H_R3.evolve([row[-1] for row in psi_t2],0.0,times3)
psi_t4 = H_R4.evolve([row[-1] for row in psi_t3],0.0,times4)

#print(np.abs(np.dot(psi_ground,np.array([row[-1] for row in psi_t1]))))
#print(np.abs(np.dot(np.array([row[-1] for row in psi_t1]),np.array([row[-1] for row in psi_t2]))))
#print(np.abs(np.dot(np.array([row[-1] for row in psi_t2]),np.array([row[-1] for row in psi_t3]))))
H_target.expt_value([row[-1] for row in psi_t4])

(0.09999972475591133+0j)

In [136]:
Gauss_law.expt_value(psi_ini)

0j